In [1]:
#importamos las librerias necesarias
import pickle
import numpy as np
import pandas as pd


In [2]:
import pandas as pd

# Cargar el archivo pickle (.pkl)
file_path = '/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/business.pkl'
df_bussines_pkl = pd.read_pickle(file_path)

In [4]:
#analizamos el peso del archivo original
import os
print("Tamaño del archivo original: ", os.path.getsize(file_path)/1e6, "MB")

Tamaño del archivo original:  116.402361 MB


In [5]:
df_bussines_pkl.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#hacer un csv para ver la estructura del archivo
df_bussines_pkl.to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/df_bussines_pkl.csv', index=False)

In [7]:
df_bussines_pkl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [13]:
df_bussines_pkl.shape

(150346, 28)

In [8]:
#mostramos la cantidad de valores nulos
print(df_bussines_pkl.isnull().sum())

business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64


In [ ]:
df_bussines_pkl.columns

In [13]:
#como vemos en estas dos ultimas celdas, encontramos un error en el que hay 14 columnas duplicadas, antes de eliminarlas, vamos a ver los valores de estas columnas, ya que solo son 5 filas

# Crear una máscara booleana para las filas que no son nulas en las columnas duplicadas
mask = df_bussines_pkl.iloc[:, 14:].notnull().any(axis=1)

# Aplicar la máscara al DataFrame para ver solo las filas que no son nulas en las columnas duplicadas
df_bussines_pkl[mask]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
590,6_Zf2QsRHCH1RhbzA9inyw,Starbucks,8602 Hillsborough Ave,Tampa,LA,33615,27.99637,-82.580954,3.0,56,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Food","{'Monday': '5:30-21:0', 'Tuesday': '5:30-21:0'...",6_Zf2QsRHCH1RhbzA9inyw,STarbucks,8602 Hillsborough Ave,Tampa,LA,33615,27.99637,-82.580954,3.0,56,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Coffee & Tea, Food","{'Monday': '5:30-21:0', 'Tuesday': '5:30-21:0'..."
613,Iipfa9Kaac9XH5Vo8GQwvg,Starbucks,5956 153 Avenue,Edmonton,PA,T5Y 2W1,53.616147,-113.432424,3.5,10,1,"{'BusinessParking': '{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '5:30-22:0', 'Tuesday': '5:30-22:0'...",Iipfa9Kaac9XH5Vo8GQwvg,STarbucks,5956 153 Avenue,Edmonton,PA,T5Y 2W1,53.616147,-113.432424,3.5,10,1,"{'BusinessParking': '{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '5:30-22:0', 'Tuesday': '5:30-22:0'..."
978,oY6osC4Ch3PvguA7D1hh_w,Starbucks,1251 Route 130 S,Cinnaminson,AZ,08077,40.000157,-74.987313,4.5,7,1,"{'WiFi': 'u'free'', 'BusinessParking': '{'gara...","Coffee & Tea, Food","{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'...",oY6osC4Ch3PvguA7D1hh_w,STarbucks,1251 Route 130 S,Cinnaminson,AZ,08077,40.000157,-74.987313,4.5,7,1,"{'WiFi': 'u'free'', 'BusinessParking': '{'gara...","Coffee & Tea, Food","{'Monday': '5:30-20:0', 'Tuesday': '5:30-20:0'..."
1296,pxuMxh5oC7W-PDN9wwvozg,Starbucks,219 N Center St,Reno,FL,89501,39.527772,-119.812518,3.0,44,0,"{'BikeParking': 'False', 'Caters': 'False', 'O...","Coffee & Tea, Food","{'Monday': '5:30-19:0', 'Tuesday': '5:30-19:0'...",pxuMxh5oC7W-PDN9wwvozg,STarbucks,219 N Center St,Reno,FL,89501,39.527772,-119.812518,3.0,44,0,"{'BikeParking': 'False', 'Caters': 'False', 'O...","Coffee & Tea, Food","{'Monday': '5:30-19:0', 'Tuesday': '5:30-19:0'..."
1543,lo2VbA7Lwzu3i23pQpmzlw,Starbucks,"1125 S Black Horse Pike, Space 575",Gloucester Township,NJ,08012,39.78403,-75.05207,3.0,15,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Food, Coffee & Tea","{'Monday': '0:0-0:0', 'Tuesday': '10:30-20:0',...",lo2VbA7Lwzu3i23pQpmzlw,STarbucks,"1125 S Black Horse Pike, Space 575",Gloucester Township,NJ,08012,39.78403,-75.05207,3.0,15,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Food, Coffee & Tea","{'Monday': '0:0-0:0', 'Tuesday': '10:30-20:0',..."


In [14]:
#como vemos, los unicos 5 valores que encontramos en las 14 columnas duplicadas justamente DUPLICAN lo que ya esta en las columnas originales, por lo que procedemos a eliminarlas

# pero antes, y COMO SIEMPRE, vamos a guardar una copia del archivo original

df_bussines_pkl_sin_col_duplicadas = df_bussines_pkl.copy()

In [16]:
#verifico que se haya guardado correctamente en crudo:
df_bussines_pkl_sin_col_duplicadas.shape

(150346, 28)

In [17]:
# y ahora si, Eliminamos las columnas duplicadas del DataFrame copia
df_bussines_pkl_sin_col_duplicadas = df_bussines_pkl.iloc[:, :14]

In [18]:
#y vuelvo a verificar: 
df_bussines_pkl_sin_col_duplicadas.shape

(150346, 14)

In [19]:
#vuelvo a analizare los valores nulos
print(df_bussines_pkl_sin_col_duplicadas.isnull().sum())

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64


### Estudio de las columnas con nulos: 

<u>attributes:</u> Es un conjunto de atributos adicionales del negocio. Puede ser una cadena de texto, un diccionario u otro tipo de dato que contenga información adicional como si el negocio tiene Wi-Fi, accesibilidad para discapacitados, etc. <span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u>categories:</u> Son las categorías a las que pertenece el negocio. Es una cadena de texto que puede contener múltiples categorías separadas por comas, como "restaurante, italiano, pizza".<span style='color:lightblue'>consideramos que puede ser necesaria a nuestros intereses</span><span style='color:Gold'> SE ANALIZARA COMO RELLENAR LOS NULOS(103)</span>

<u>hours:</u> Es el horario de apertura del negocio. Puede ser una cadena de texto, un diccionario u otro tipo de dato que especifique los horarios de apertura y cierre para cada día de la semana. <span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

In [20]:
#Dropeamos las columnas 'attributes' y 'hours' de acuerdo al analisis previo:

df_bussines_pkl_sin_col_duplicadas = df_bussines_pkl_sin_col_duplicadas.drop(['attributes', 'hours'], axis=1)

In [21]:
#verificamos que se hayan eliminado correctamente
df_bussines_pkl_sin_col_duplicadas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  categories    150243 non-null  object
dtypes: object(12)
memory usage: 14.9+ MB


In [24]:
#miramos los valores nulos de las filas en la columna 'categories'

# Crear una máscara booleana para las filas que son nulas en la columna 'categories'
mask = df_bussines_pkl_sin_col_duplicadas['categories'].isnull()

# Aplicar la máscara al DataFrame para ver solo las filas que son nulas en la columna 'categories'
df_bussines_pkl_sin_col_duplicadas[mask]

#y guardamos los datos de la mascara en un archivo csv para analizarlo mejor

df_bussines_pkl_sin_col_duplicadas[mask].to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/df_bussines_analizar_categorias_nulas.csv', index=False)



In [25]:
#guardamos el archivo limpio

df_bussines_pkl_sin_col_duplicadas.to_pickle('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/business_listo_para_usar.pkl')

In [28]:
#y guardamos un csv para ver la estructura del archivo final

df_bussines_pkl_sin_col_duplicadas.to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/bussines_pkl/df_bussines_listo_para_usar.csv', index=False)


In [27]:
df_bussines_pkl_sin_col_duplicadas.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories'],
      dtype='object')